<h1 align="center">Caixa Geral de Depósitos Overview</h1>

A notebook that allows for easy import and easy analysis through python on your CGD account balance.

In [ ]:
import pandas as pd
from math import isnan
import matplotlib.pyplot as plt

# Data gathering instructions
 * Go to Dashboard
 * "Contas à ordem" -> "Saldos e Movimentos"
 * Select the dates between which you want to export
 * Upper right corner -> "Gravar em XLS"
 * Place the `.csv` document in the same folder as this notebook and run the cells
 * The document should be named `conta.csv`
 * Done

# Reading from csv
(explanation)
 * Exclude first 6 rows
 * Use special encoding
 * Separator is `;` instead of `,`
 * Remove last two lines, which are just Totals
 * Drop last column, which is just ",,,"

In [ ]:
def replace_numeric_column(df, name):
    df[name] = df[name].apply(lambda x: str(x).replace(".","").replace(",","."))
    df[name] = pd.to_numeric(df[name], errors='coerce')

In [ ]:
types={}
df = pd.read_csv("conta.csv", header=6, encoding="ISO-8859-1", sep=";", decimal=",", thousands=".")
df.columns = df.columns.str.replace(" ", "")
df.drop(df.tail(2).index,inplace=True) # instead of skipfooter (which cannot use C) just remove last 2 rows
df = df.drop([",,,"], axis=1)

replace_numeric_column(df, 'Débito')
replace_numeric_column(df, 'Crédito')
replace_numeric_column(df, 'SaldoContab.')
replace_numeric_column(df, 'Saldodisponível')
df.head()

### Merge Debit and Credit
Merge into one column with positive and negative balances

In [ ]:
def get_montante(deb, cred):
    if isnan(float(deb)):
        return cred
    else:
        return -float(deb)
df["Montante"] = df.apply(lambda row: get_montante(row['Débito'], row['Crédito']), axis=1);

In [ ]:
df = df.drop(['Débito', 'Crédito'], axis=1)
df.head()

In [ ]:
pd.to_datetime(df['Datamov.']);
pd.to_datetime(df['Datavalor']);

## Plot Account Movements

In [ ]:
def plot_reverse(tdf):
    tdf.plot(figsize=(20,10));
    plt.gca().invert_xaxis()
    plt.suptitle("Overall Evolution", fontsize=30);
    tdf.drop(columns=["SaldoContab.","Saldodisponível"]).plot(figsize=(10,5),color=["g"]);
    plt.gca().invert_xaxis()
    plt.suptitle("Montante evolution", fontsize=30);

In [ ]:
plot_reverse(df)

## Higest Withdrawals

In [ ]:
df.sort_values(by=["Montante"]).head(10)

## Higest Deposits

In [ ]:
df.sort_values(by=["Montante"],ascending=False).head(10)

## Search Query
regex is accepted: `query="deposit.*soft"`

In [ ]:
def perform_query(sender):
    query = sender.value # get the input value
    matches = df[df['Descrição'].str.contains(query,case=False,regex=True)==True]
    print("Found: %d matches" % len(matches))
    print("Total moved:     %.2f" % matches["Montante"].abs().sum())
    print("Total withdrawn: %.2f" % matches[matches.Montante <0].Montante.abs().sum())
    print("Total deposited: %.2f" % matches[matches.Montante >0].Montante.sum())
    plot_reverse(matches) # show the plot
    display(matches) # display the dataframe

In [ ]:
from ipywidgets import widgets, Layout
q = widgets.Text(
    value='',
    placeholder='Your regex search query (you can use any regex feature, try: ".*")',
    description='Search for:',
    layout=Layout(width='60%')
)
q.on_submit(perform_query)
display(q)

## Further analysis
Perform more analysis on `df` as you please

In [ ]:
df.head()